# Practical Exam: Loan Insights

EasyLoan offers a wide range of loan services, including personal loans, car loans, and mortgages.

EasyLoan offers loans to clients from Canada, United Kingdom and United States.

The analytics team wants to report performance across different geographic areas. They aim to identify areas of strength and weakness for the business strategy team.

They need your help to ensure the data is accessible and reliable before they start reporting.


**Database Schema**

The data you need is in the database named `lending`.

![database schema](lending_schema.png)

# Task 1 

The analytics team wants to use the `client` table to create a dashboard for client details. For them to proceed, they need to be sure the data is clean enough to use.

The `client` table below illustrates what the analytics team expects the data types and format to be.

Write an SQL query that returns the `client` table with the specified format, including identifying and cleaning all invalid values. 
-  Your output should be a DataFrame with the name 'client'. Do not modify the `client` table.
-  Note that the DataLab environment formats dates as YYYY-MM-DD-hh-ss-SSS. 

| Column Name       | Description                                                      |
|-------------------|------------------------------------------------------------------|
| client_id         | Unique integer (set by the database, can’t take any other value) |
| date\_of\_birth       | Date of birth of the client, as a date                           |
| employment_status        | Current employment status of the client, either employed or unemployed, as a lower case string                              |
| country          | The country where the client resides, either USA, UK or CA, as an upper case string                      |

In [ ]:
from pathlib import Path
import sqlite3
import pandas as pd

db_path = next(
    (p for p in ("lending.db", "lending.sqlite", "lending.sqlite3", "lending") if Path(p).exists()),
    None,
)
if db_path is None:
    raise FileNotFoundError("Could not find lending database file.")
conn = sqlite3.connect(db_path)

query = """
WITH cleaned AS (
  SELECT
    client_id,
    DATE(substr(date_of_birth, 1, 10)) AS date_of_birth,
    LOWER(REPLACE(REPLACE(REPLACE(TRIM(employment_status), ' ', ''), '-', ''), '.', '')) AS employment_status_clean,
    UPPER(REPLACE(REPLACE(REPLACE(TRIM(country), ' ', ''), '-', ''), '.', '')) AS country_clean
  FROM client
)
SELECT
  client_id,
  date_of_birth,
  CASE
    WHEN employment_status_clean IN ('employed','employee','employeed','selfemployed','selfemployeed') THEN 'employed'
    WHEN employment_status_clean IN ('unemployed','unemployeed','unemployd','unemploy') THEN 'unemployed'
    ELSE NULL
  END AS employment_status,
  CASE
    WHEN country_clean IN ('USA','US','UNITEDSTATES','UNITEDSTATESOFAMERICA') THEN 'USA'
    WHEN country_clean IN ('UK','UNITEDKINGDOM','GREATBRITAIN') THEN 'UK'
    WHEN country_clean IN ('CA','CANADA') THEN 'CA'
    ELSE NULL
  END AS country
FROM cleaned;
"""

client = pd.read_sql_query(query, conn)
client

NameError: name 'conn' is not defined

# Task 2

You have been told that there was a problem in the backend system as some of the `repayment_channel` values are missing. 

The missing values are critical to the analysis so they need to be filled in before proceeding.

Luckily, they have discovered a pattern in the missing values:

- Repayment higher than 4000 dollars should be made via `bank account`.
- Repayment lower than 1000 dollars should be made via `mail`.

Write an SQL query that makes the `repayment` table match these criteria.
-  Your output should be a DataFrame with the name 'repayment'. Do not modify the original `repayment` table.

In [ ]:
from pathlib import Path
import sqlite3
import pandas as pd

db_path = next(
    (p for p in ("lending.db", "lending.sqlite", "lending.sqlite3", "lending") if Path(p).exists()),
    None,
)
if db_path is None:
    raise FileNotFoundError("Could not find lending database file.")
conn = sqlite3.connect(db_path)

query = """
SELECT
  repayment_id,
  loan_id,
  repayment_amount,
  repayment_date,
  CASE
    WHEN repayment_channel IS NOT NULL THEN repayment_channel
    WHEN repayment_amount > 4000 THEN 'bank account'
    WHEN repayment_amount < 1000 THEN 'mail'
    ELSE NULL
  END AS repayment_channel
FROM repayment;
"""

repayment = pd.read_sql_query(query, conn)
repayment

NameError: name 'conn' is not defined

# Task 3

Starting on January 1st, 2022, all US clients started to use an online system to sign contracts.

The analytics team wants to analyze the loans for US clients who used the new online system.

Write a query that returns the data for the analytics team. Your output should include `client_id`,`contract_date`, `principal_amount` and `loan_type` columns.

![database schema](lending_schema.png)

In [ ]:
from pathlib import Path
import sqlite3
import pandas as pd

db_path = next(
    (p for p in ("lending.db", "lending.sqlite", "lending.sqlite3", "lending") if Path(p).exists()),
    None,
)
if db_path is None:
    raise FileNotFoundError("Could not find lending database file.")
conn = sqlite3.connect(db_path)

query = """
SELECT
  l.client_id,
  l.contract_date,
  l.principal_amount,
  lt.loan_type
FROM loan AS l
JOIN client AS c
  ON c.client_id = l.client_id
JOIN loan_type AS lt
  ON lt.loan_type_id = l.loan_type_id
WHERE UPPER(TRIM(c.country)) IN ('USA', 'US')
  AND DATE(substr(l.contract_date, 1, 10)) >= '2022-01-01';
"""

us_loans = pd.read_sql_query(query, conn)
us_loans

NameError: name 'conn' is not defined

# Task 4

The business strategy team is considering offering a more competitive rate to the US market. 

The analytic team want to compare the average interest rates offered by the company for the same loan type in different countries to determine if there are significant differences.

Write a query that returns the data for the analytics team. Your output should include `loan_type`, `country` and `avg_rate` columns.

![database schema](lending_schema.png)

In [ ]:
from pathlib import Path
import sqlite3
import pandas as pd

db_path = next(
    (p for p in ("lending.db", "lending.sqlite", "lending.sqlite3", "lending") if Path(p).exists()),
    None,
)
if db_path is None:
    raise FileNotFoundError("Could not find lending database file.")
conn = sqlite3.connect(db_path)

query = """
SELECT
  lt.loan_type,
  UPPER(TRIM(c.country)) AS country,
  AVG(l.interest_rate) AS avg_rate
FROM loan AS l
JOIN client AS c
  ON c.client_id = l.client_id
JOIN loan_type AS lt
  ON lt.loan_type_id = l.loan_type_id
GROUP BY
  lt.loan_type,
  UPPER(TRIM(c.country));
"""

average_data = pd.read_sql_query(query, conn)
average_data

NameError: name 'conn' is not defined